# Import

In [ ]:
# Creation of the Excel file
# Terminal colors

def color_print(msg, color=Fore.CYAN, style=Style.NORMAL):
    """ Print a colored message """
    print(color + style + msg + Style.RESET_ALL)

def acquire_data(dataset_paths, shape_ext, output_path):
    color_print("\n--- Step 1. Acquire Data ---", Fore.GREEN, Style.BRIGHT)
    os.makedirs(output_path, exist_ok=True)

    shape_filenames = []
    dataset_ids = []

    for data_path, dataset_id in dataset_paths:
        files = sorted(glob.glob(os.path.join(data_path, '*' + shape_ext)))
        shape_filenames.extend(files)
        dataset_ids.extend([dataset_id] * len(files))

    color_print(f"  Number of shapes : {len(shape_filenames)}", Fore.YELLOW)

    return shape_filenames, dataset_ids

def generate_shapes(real_shapes, num_particles, n_new=10, n_components=10, variation_scale=1.0):
    """
    Generates new flattened shapes (nb_subjects, features) by perturbing the principal components.

    Params:
        real_shapes (np.ndarray): array (n_samples, n_points, 3) or (n_samples, dim)
        num_particles (int): number of points per shape
        n_new (int): number of shapes to generate
        n_components (int): number of principal components to use
        variation_scale (float): scale factor for the variation

    Returns:
        new_shapes (np.ndarray): new shapes of size (n_new, features)
    """
    # Convert to 2D if necessary
    # PCA
    # Mean and std of the components
    # Generation


# Label alignment

In [ ]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import center_of_mass

# Folder containing the .nii.gz files
# folder_path = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/BDD/healthy_subj_JEREMIE_RF/healthy_subj_JEREMIE_RF/JEREMIE_RF"
# folder_path = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/BDD/VL_labels/VL_young_labels/labels/VL_JEREMIE"
folder_path = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/BDD/VL_labels/VL_diasem_labels/labels/VL_Diasem/manual_gz"
save_folder = os.path.join(folder_path, "aligned")
os.makedirs(save_folder, exist_ok=True)

# Retrieve NIfTI files
nii_files = [f for f in os.listdir(folder_path) if f.endswith(".nii.gz")]
nii_files = sorted(nii_files)  # Ensure order
print(f"🔄 Processing {len(nii_files)} NIfTI files...\n")
# === 1. Compute the center of mass of the first file as reference ===
ref_file = "sujet1_VL_pad_seg.nii.gz"
print(f"Reference file: {ref_file}")
ref_path = os.path.join(folder_path, ref_file)
ref_img = nib.load(ref_path)
ref_data = ref_img.get_fdata(dtype=np.float32)
ref_affine = ref_img.affine

# Center of mass of the reference file (in mm)
ref_com_voxel = np.array(center_of_mass(ref_data))
ref_com_mm = nib.affines.apply_affine(ref_affine, ref_com_voxel)

print(f"[REF] File: {ref_file} | COM (mm): {ref_com_mm}")

# === 2. Apply recentering based on the reference to other files ===
for index, nii_file in enumerate(nii_files, start=1):
    try:
        if nii_file == ref_file:
            continue
        else:
            input_path = os.path.join(folder_path, nii_file)
            output_path = os.path.join(save_folder, nii_file.replace(".nii.gz", ".nii.gz"))
            print(f"Processing: {index}: {nii_file}")
            img = nib.load(input_path)
            # Use get_fdata(dtype=np.uint8) to limit memory usage
            data = img.get_fdata(dtype=np.float32)
            affine = img.affine.copy()
            header = img.header.copy()

            # Current center of mass
            com_voxel = np.array(center_of_mass(data))
            com_mm = nib.affines.apply_affine(affine, com_voxel)

            # Translation to apply to align with the reference COM
            translation = ref_com_mm - com_mm
            affine[:3, 3] += translation

            # Save
            new_img = nib.Nifti1Image(data.astype(np.uint8), affine, header)
            nib.save(new_img, output_path)

    except Exception as e:
        print(f"Error while processing {nii_file}: {e}")


🔄 Traitement de 59 fichiers NIfTI...

Fichier Ref : sujet1_VL_pad_seg.nii.gz
[REF] Fichier : sujet1_VL_pad_seg.nii.gz | COM (mm) : [  11.99311094   26.88136966 -464.23746054]
Traitement : 1: AL135_VL_seg.nii.gz
Traitement : 2: AM112_VL_seg.nii.gz
Traitement : 3: BA122_VL_seg.nii.gz
Traitement : 4: BA137_VL_seg.nii.gz
Traitement : 5: BC167_VL_seg.nii.gz
Traitement : 6: BH116_VL_seg.nii.gz
Traitement : 7: BJ128_VL_seg.nii.gz
Traitement : 8: BM103_VL_seg.nii.gz
Traitement : 9: BP149_VL_seg.nii.gz
Traitement : 10: CG115_VL_seg.nii.gz
Traitement : 11: CG147_VL_seg.nii.gz
Traitement : 12: CR159_VL_seg.nii.gz
Traitement : 13: CS102_VL_seg.nii.gz
Traitement : 14: DB145_VL_seg.nii.gz
Traitement : 15: DH156_VL_seg.nii.gz
Traitement : 16: DI154_VL_seg.nii.gz
Traitement : 17: DJ101_VL_seg.nii.gz
Traitement : 18: DO125_VL_seg.nii.gz
Traitement : 19: DP172_VL_seg.nii.gz
Traitement : 20: DR160_VL_seg.nii.gz
Traitement : 21: EM151_VL_seg.nii.gz
Traitement : 22: EP105_VL_seg.nii.gz
Traitement : 23: FD1

In [ ]:
import os
import nibabel as nib
import numpy as np
import math
from scipy.ndimage import center_of_mass

# === Parameters ===
folder_path = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/BDD/VL_labels/VL_diasem_labels/labels/VL_Diasem/manual_gz"
save_folder = os.path.join(folder_path, "aligned_rot180")
os.makedirs(save_folder, exist_ok=True)

# List of .nii.gz files
nii_files = [f for f in os.listdir(folder_path) if f.endswith(".nii.gz")]
nii_files = sorted(nii_files)
print(f"🔄 Processing {len(nii_files)} NIfTI files...\n")

# === Reference file ===
ref_file = "sujet1_VL_pad_seg.nii.gz"
ref_path = os.path.join(folder_path, ref_file)
ref_img = nib.load(ref_path)
ref_data = ref_img.get_fdata(dtype=np.float32)
ref_affine = ref_img.affine

# Compute the center of mass (mm) of the reference
ref_com_voxel = np.array(center_of_mass(ref_data))
ref_com_mm = nib.affines.apply_affine(ref_affine, ref_com_voxel)
print(f"[REF] File: {ref_file} | COM (mm): {ref_com_mm}")

# === Define 180° rotation around Z ===
angle_deg = 180
angle_rad = math.radians(angle_deg)
rotation_z = np.array([
    [np.cos(angle_rad), -np.sin(angle_rad), 0],
    [np.sin(angle_rad),  np.cos(angle_rad), 0],
    [0, 0, 1]
])

# === Process files ===
for index, nii_file in enumerate(nii_files, start=1):
    try:
        if nii_file == ref_file:
            continue

        input_path = os.path.join(folder_path, nii_file)
        output_path = os.path.join(save_folder, nii_file.replace(".nii.gz", ".nii.gz"))
        print(f"🧠 Processing: {index} - {nii_file}")

        # Load file
        img = nib.load(input_path)
        data = img.get_fdata(dtype=np.float32)
        affine = img.affine.copy()
        header = img.header.copy()

        # Compute current COM (mm)
        com_voxel = np.array(center_of_mass(data))
        com_mm = nib.affines.apply_affine(affine, com_voxel)

        # === Step 1: rotation around the current center of mass ===
        T_to_origin = np.eye(4)
        T_to_origin[:3, 3] = -com_mm

        R = np.eye(4)
        R[:3, :3] = rotation_z

        T_back = np.eye(4)
        T_back[:3, 3] = com_mm

        transform = T_back @ R @ T_to_origin

        # New affine after rotation around the COM
        affine = transform @ affine

        # === Step 2: align the COM with the reference COM ===
        com_mm_new = nib.affines.apply_affine(affine, com_voxel)
        translation = ref_com_mm - com_mm_new
        affine[:3, 3] += translation

        # === Save ===
        new_img = nib.Nifti1Image(data.astype(np.uint8), affine, header)
        nib.save(new_img, output_path)

    except Exception as e:
        print(f"❌ Error while processing {nii_file}: {e}")



🔄 Traitement de 58 fichiers NIfTI...

[REF] Fichier : sujet1_VL_pad_seg.nii.gz | COM (mm) : [ -89.0210721   -71.57960421 -230.51070841]
🧠 Traitement : 1 - AL135_VL.nii.gz
🧠 Traitement : 2 - AM112_VL.nii.gz
🧠 Traitement : 3 - BA122_VL.nii.gz
🧠 Traitement : 4 - BA137_VL.nii.gz
🧠 Traitement : 5 - BC167_VL.nii.gz
🧠 Traitement : 6 - BH116_VL.nii.gz
🧠 Traitement : 7 - BJ_128VL.nii.gz
🧠 Traitement : 8 - BM103_VL.nii.gz
🧠 Traitement : 9 - BP149_VL.nii.gz
🧠 Traitement : 10 - CG115_VL.nii.gz
🧠 Traitement : 11 - CG147_VL.nii.gz
🧠 Traitement : 12 - CR159_VL.nii.gz
🧠 Traitement : 13 - CS102_VL.nii.gz
🧠 Traitement : 14 - DB145_VL.nii.gz
🧠 Traitement : 15 - DH156_VL.nii.gz
🧠 Traitement : 16 - DI154_VL.nii.gz
🧠 Traitement : 17 - DJ101_VL.nii.gz
🧠 Traitement : 18 - DO125_VL.nii.gz
🧠 Traitement : 19 - DP172_VL.nii.gz
🧠 Traitement : 20 - DR160_VL.nii.gz
🧠 Traitement : 21 - EM151_VL.nii.gz
🧠 Traitement : 22 - EP105_VL.nii.gz
🧠 Traitement : 23 - FD121_VL.nii.gz
🧠 Traitement : 24 - FM144_VL.nii.gz
🧠 Traitem

# Particles files selection

In [ ]:
import os
import shutil

# Paths
source_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J_128_RF4_particles"
sarcopenie_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/Sarcopenie_128"
ages_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/Ages_128"
jeunes_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/Jeunes_128"
malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"

# Create target folders if they do not exist
os.makedirs(sarcopenie_folder, exist_ok=True)
os.makedirs(ages_folder, exist_ok=True)
os.makedirs(jeunes_folder, exist_ok=True)

# Load patient codes (XXXX)
with open(malade_list_file, "r") as f:
    malades = set(line.strip() for line in f if line.strip())

# Browse files
for file in os.listdir(source_folder):
    if file.endswith("world.particles") and file.startswith("Malades_"):
        parts = file.split("_")
        print(parts)
        if len(parts) >= 2:
            code = parts[1]
            if code in malades:
                src_path = os.path.join(source_folder, file)
                dst_path = os.path.join(sarcopenie_folder, file)
                shutil.copy(src_path, dst_path)  # or shutil.copy() if you want to copy
                print(f"✅ File moved: {file}")
            else:
                src_path = os.path.join(source_folder, file)
                dst_path = os.path.join(ages_folder, file)
                shutil.copy(src_path, dst_path)  # or shutil.copy() if you want to copy
                print(f"✅ File moved: {file}")
    if file.endswith("world.particles") and file.startswith("Jeunes_"):
        src_path = os.path.join(source_folder, file)
        dst_path = os.path.join(jeunes_folder, file)
        shutil.copy(src_path, dst_path)  # or shutil.copy() if you want to copy
        print(f"✅ File moved: {file}")

print("Done.")


✅ Fichier déplacé : Jeunes_sujet10_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet10_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet10_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet11_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet11_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet11_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet12_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet12_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet12_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet13_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet13_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet13_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet14_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet14_RF_fort_3DUS_label_world.particles
✅ Fichier dépl

In [ ]:
import os
import shutil

# Paths
# RF
source_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J__particles"
train_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/train5"
test_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/test5"
test_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/test_cases_5.txt"
train_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/train_cases_5.txt"

sarcopenie_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/Sarcopenie_128"
ages_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/Ages_128"
jeunes_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/Jeunes_128"

# TA
# source_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/D_J_128_TA_particles"
# train_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/train"
# test_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/test"
# test_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/test_cases_TA.txt"
# train_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/train_cases_TA.txt"

# sarcopenie_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/Sarcopenie_128"
# ages_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/Ages_128"
# jeunes_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/Jeunes_128"

# #VL
# source_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/D_J_128_VL_particles"
# train_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/train"
# test_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/test"
# test_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/test_cases_1.txt"
# train_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/train_cases_1.txt"

# sarcopenie_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/Sarcopenie_128"
# ages_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/Ages_128"
# jeunes_folder = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/Jeunes_128"


malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"

# Create target folder if it does not exist
os.makedirs(sarcopenie_folder, exist_ok=True)
os.makedirs(ages_folder, exist_ok=True)
os.makedirs(jeunes_folder, exist_ok=True)
# Create target folder if it does not exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Load patient codes (XXXX)
with open(test_list_file, "r") as f:
    tests = set(line.strip() for line in f if line.strip())
with open(train_list_file, "r") as f:
    trains = set(line.strip() for line in f if line.strip())
# Load patient codes (XXXX)
with open(malade_list_file, "r") as f:
    malades = set(line.strip() for line in f if line.strip())

# Browse files
for file in os.listdir(source_folder):
    if file.endswith("world.particles") and file.startswith("Malades_"):
        parts = file.split("_")
        print(parts)
        if len(parts) >= 2:
            code = parts[1]
            if code in malades:
                src_path = os.path.join(source_folder, file)
                dst_path = os.path.join(sarcopenie_folder, file)
                shutil.copy(src_path, dst_path)  # or shutil.copy() if you want to copy
                print(f"✅ File moved: {file}")
            else:
                src_path = os.path.join(source_folder, file)
                dst_path = os.path.join(ages_folder, file)
                shutil.copy(src_path, dst_path)  # or shutil.copy() if you want to copy
                print(f"✅ File moved: {file}")
    if file.endswith("world.particles") and file.startswith("Jeunes_"):
        src_path = os.path.join(source_folder, file)
        dst_path = os.path.join(jeunes_folder, file)
        shutil.copy(src_path, dst_path)  # or shutil.copy() if you want to copy
        print(f"✅ File moved: {file}")
print(os.listdir(source_folder))
print(f"\n {trains}")
for train in trains:
    for file in os.listdir(source_folder):
        if file.endswith("world.particles") and train in file:
            chemin_source = os.path.join(source_folder, file)
            chemin_dest = os.path.join(train_folder, file)
            shutil.copy(chemin_source, chemin_dest)
            print(f"✅ File moved to train: {file}")

for test in tests:
    for file in os.listdir(source_folder):
        if file.endswith("world.particles") and test in file:
            chemin_source = os.path.join(source_folder, file)
            chemin_dest = os.path.join(test_folder, file)
            shutil.copy(chemin_source, chemin_dest)
            print(f"✅ File moved to test: {file}")


print("Done.")



✅ Fichier déplacé : Jeunes_sujet10_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet10_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet10_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet11_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet11_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet11_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet12_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet12_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet12_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet13_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet13_RF_fort_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet13_RF_pad_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet14_RF_faible_3DUS_label_world.particles
✅ Fichier déplacé : Jeunes_sujet14_RF_fort_3DUS_label_world.particles
✅ Fichier dépl

# Add LDA parameters to Shapeworks

In [ ]:
import json

# === Parameters ===
# input_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J_128_RF6.swproj"
# output_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J_128_RF_LDA.swproj"
# malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"
# input_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/D_J_128_TA.swproj"
# output_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/D_J_128_RF_LDA_sains_vs_sarco.swproj"
# malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"
fichier_entree = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/D_J_128_VL.swproj"
fichier_sortie = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/D_J_128_VL_LDA_sains_vs_sarco.swproj"
malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"

# List of patient file names (adapt as needed)
# Load patient codes (XXXX)
with open(malade_list_file, "r") as f:
    fichiers_malades = set(line.strip() for line in f if line.strip())

# === Load the swproj file ===
with open(fichier_entree, "r") as f:
    projet = json.load(f)

# === Add groups in each entry of the 'data' list ===
for sujet in projet.get("data", []):
    nom = sujet.get("name", "")
    parts = nom.split("_")
    # if len(parts) >= 2:
    code = parts[0]
    if code in fichiers_malades:
        print(nom)
        sujet["group_ids"] = "sarcopenie"
    # elif code not in fichiers_malades and code.lower().startswith("sujet"):
    #     sujet["group_ids"] = "jeunes"
    # else:
    #     sujet["group_ids"] = "ages"
    elif code not in fichiers_malades and code.lower().startswith("sujet"):
        sujet["group_ids"] = "sains"
    else:
        sujet["group_ids"] = "sains"

# === Save the new file ===
with open(fichier_sortie, "w") as f:
    json.dump(projet, f, indent=4)

print(f"Modified file saved as: {fichier_sortie}")


AL135_VL.nii
AM112_VL.nii
BA137_VL.nii
BM103_VL.nii
DI154_VL.nii
DP172_VL.nii
DR160_VL.nii
EM151_VL.nii
EP105_VL.nii
FO117_VL.nii
GD171_VL.nii
LA134_VL.nii
LA162_VL.nii
LJ139_VL.nii
LP133_VL.nii
MA119_VL.nii
MM170_VL.nii
PG161_VL.nii
PP104_VL.nii
QP108_VL.nii
RA150_VL.nii
RJ136_VL.nii
SC146_VL.nii
Fichier modifié sauvegardé sous : C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/D_J_128_VL_LDA_sains_vs_sarco.swproj


## Healthy vs Sarcopenia

In [ ]:
import json

# === Parameters ===
input_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J_128_RF6.swproj"
output_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J_128_RF_LDA_sains_vs_sarco.swproj"
malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"
# input_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/D_J_128_TA.swproj"
# output_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_TA/Run_1/OUTPUT/D_J_128_RF_LDA.swproj"
# malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"
# input_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/D_J_128_VL.swproj"
# output_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_VL_test30/Run_1/OUTPUT/D_J_128_VL_LDA.swproj"
# malade_list_file = "C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/sarcopenia_subj_DIASEM.txt"

# List of patient file names (adapt as needed)
# Load patient codes (XXXX)
with open(malade_list_file, "r") as f:
    fichiers_malades = set(line.strip() for line in f if line.strip())


# === Load the swproj file ===
with open(input_file, "r") as f:
    projet = json.load(f)


# === Add groups in each entry of the 'data' list ===
for sujet in projet.get("data", []):
    nom = sujet.get("name", "")
    parts = nom.split("_")
    # if len(parts) >= 2:
    code = parts[0]
    if code in fichiers_malades:
        print(nom)
        sujet["group_ids"] = "sarcopenie"
    elif code not in fichiers_malades and code.lower().startswith("sujet"):
        sujet["group_ids"] = "normal"
    else:
        sujet["group_ids"] = "normal"

# === Save the new file ===
with open(output_file, "w") as f:
    json.dump(projet, f, indent=4)

print(f"Modified file saved as: {output_file}")


AL135_RF.nii
AM112_RF_label.nii
BA137_RF_label.nii
BM103_RF_label.nii
CY164_RF_seg.nii
DI154_RF_label.nii
DP172_RF_label.nii
DR160_RF_label.nii
EM151_RF_label.nii
EP105_RF_label.nii
FO117_RF_label.nii
GD171_RF_label.nii
LA134_RF_label.nii
LA162_RF_label.nii
LJ139_RF_label.nii
LP133_RF_label.nii
MA119_RF_label.nii
MM170_RF_label.nii
PG161_RF_label.nii
PP104_RF_label.nii
QP108_RF_label.nii
RA150_RF_label.nii
RJ136_RF_label.nii
SC146_RF_label.nii
Fichier modifié sauvegardé sous : C:/Users/gossa/OneDrive/Documents/Travail INSA Lyon/PFE/Code/D_J_128_RF6/Run_1/OUTPUT/D_J_128_RF_LDA_sains_vs_sarco.swproj
